In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Vipin Vashisth\\Desktop\\ML_Projects\\MLflow_Implementation\\MLFlow-End-to-End-implementation\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Vipin Vashisth\\Desktop\\ML_Projects\\MLflow_Implementation\\MLFlow-End-to-End-implementation'